<a href="https://colab.research.google.com/github/raj075512/equity--resarcher-project/blob/main/financeGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install datasets loralib sentencepiece

!pip -q install langchain_community
!pip -q install unstructured
!pip -q install streamlit
!pip -q install sentence_transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
!pip -q install huggingface_hub
!pip -q install openai
!pip -q install tiktoken
!pip -q install faiss-cpu
!pip -q install tokenizers
!pip -q install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 2.2 MB/s eta 0:00:00


In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [ ]:

import os
import streamlit as st
import torch
import sys
import pickle

from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings


from langchain.vectorstores import FAISS


from transformers import  ( AutoTokenizer, AutoModelForCausalLM ,pipeline ,
                              BitsAndBytesConfig,
                            )
from langchain.llms import HuggingFacePipeline



In [ ]:
from huggingface_hub import notebook_login


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

**LOAD THE URLS OR SOURCE DATA**

In [ ]:
urls=['https://lmsys.org/blog/2023-03-30-vicuna/']
!pip install unstructured

from langchain_community.document_loaders import UnstructuredURLLoader
loader=UnstructuredURLLoader(urls=urls)
documents=loader.load();

len(documents )

1

**SPLIT THE TEXT INTO SMALL CHUNKS**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter(
    separators=["\n\n","\n"," "],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,

    )


In [ ]:

chunks=splitter.split_documents(documents)

len(chunks)

In [ ]:
chunks[0]

In [ ]:
chunks[1]

**VECTOR _DATABASES STORES after the embedding**

In [ ]:
!pip install faiss-cpu
!pip install sentence-transformers

from langchain.embeddings import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

FAISS - SMALL LIGHTWIGHT

In [ ]:
from langchain.vectorstores import FAISS
vectorstore=FAISS.from_documents(chunks,embeddings)








In [ ]:

from huggingface_hub import notebook_login
notebook_login()
from langchain import HuggingFacePipeline

In [ ]:
# query_result=embeddings.embed_query("what will be the next election")
# len(query_result)

In [ ]:
# import gc

# gc.collect()
# torch.cuda.empty_cache()

**llama model and tokenizer import from huggingface**

In [ ]:


from transformers import  ( AutoTokenizer, AutoModelForCausalLM ,pipeline ,
                              BitsAndBytesConfig,
                            )
import torch

model_name= "NousResearch/Llama-2-7b-chat-hf"


bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

model = AutoModelForCausalLM.from_pretrained(
        model_name,

        use_auth_token=True,
        device_map='auto',
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )

tokenizer=AutoTokenizer.from_pretrained(model_name, use_auth_token=True,)

In [ ]:

from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )


llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


In [ ]:
llm.predict("hi , how are u ?")

In [ ]:
file_path="vector_idx.pkl"
with open(file_path,"wb") as f:
  pickle.dump(vectorstore,f)


In [ ]:
if os.path.exists(file_path):
  with open(file_path,"rb") as f:
    vectorstore=pickle.load(f)


In [ ]:
chain= RetrievalQAWithSourcesChain.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())


In [ ]:
query="what about vicuna ?"
longchain.debug=True

result=chain({"query":query}, return_only_outputs=True)
result

In [ ]:
# import streamlit as st

# st.title("news article search ")
# st.sidebar.title("news headline articles")

# urls=[]

# for i in range(3):
#   urls.append(st.sidebar.text_input(f"URL{i+1}"))

# process_url_clicked=st.sidebar.button("process URLS")


# llm=OpenAI(temperature=0.9, max_tokens=500)
# file_name="faiss_store_openai.pkl"

# main_placefolder=st.empty()

# if process_url_clicked:
#   unstructured_loader=UnstructuredURLLoader(urls=urls)
#   main_placefolder.text("loading documents")
#   documents=unstructured_loader.load()

#   splitter=recursive_text_splitter=RecursiveCharacterTextSplitter(
#     separators=["\n\n","\n","."],
#     chunk_size=1000,
#     chunk_overlap=5,
#     length_function=len,

#     )
#   main_placefolder.text("splitting documents")
#   chunks=splitter.split_documents(documents)

#   embeddding=OpenAIEmbeddings()
#   vectorstore=FAISS.from_documents(chunks,embeddding)


#   with open(file_path,"wb") as f:
#     pickle.dump(vectorstore,f)



# query=main_placefolder.text_input("question : ")

# if query:
#   if os.path.exists(file_path):
#     with open(file_path,"rb") as f:
#       vectorstore=pickle.load(f)
#       chain= RetrievalQAWithSourcesChain.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())
#       result=chain({"question":query}, return_only_outputs=True)
#       st.header("answer")
#       st.subheader(result["answer"])




